<a href="https://colab.research.google.com/github/Projekt-R-DU/prva-faza/blob/master/Cnn/cnn_cifar512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import floor
from torch import nn

class ConvolutionalModel(nn.Module):

    def __init__(self, in_channels, in_width, conv1_channels, pool1_width, conv2_channels, pool2_width, fc3_width, fc4_width, class_count):
        super(ConvolutionalModel, self).__init__()
                                                                                                                # in_channels x in_width x in_width
        self.conv1 = nn.Conv2d(in_channels, conv1_channels, kernel_size=5, stride=1, padding=2, bias=True)      # conv1_channels x in_width x in_width
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(pool1_width, stride=2)                                                        # conv1_channels x w2 x w2

        w2 = floor((in_width - pool1_width) / 2 + 1)

        self.conv2 = nn.Conv2d(conv1_channels, conv2_channels, kernel_size=5, stride=1, padding=2, bias=True)   # conv2_channels x w2 x w2
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(pool2_width, stride=2)                                                        # conv2_channels x w3 x w3
        
        w3 = floor((w2 - pool2_width) / 2 + 1)

        self.flatten3 = nn.Flatten()                                                                            # (conv2_channels x w3 x w3)
        self.fc3 = nn.Linear((int)(conv2_channels * w3 * w3), fc3_width)                                        # fc3width
        self.relu3 = nn.ReLU()

        self.fc4 = nn.Linear(fc3_width, fc4_width)                                                              # fc4width
        self.relu4 = nn.ReLU()                                                       

        self.fc_logits = nn.Linear(fc4_width, class_count)                                                      # class_count

        self.reset_parameters()

    def reset_parameters(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear) and m is not self.fc_logits:
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)
        self.fc_logits.reset_parameters()

    def forward(self, x):
        h = self.conv1(x)
        h = self.relu1(h)
        h = self.pool1(h)

        h = self.conv2(h)
        h = self.relu2(h)
        h = self.pool2(h)
        
        h = self.flatten3(h)
        h = self.fc3(h)
        h = self.relu3(h)

        h = self.fc4(h)
        h = self.relu4(h)

        logits = self.fc_logits(h)
        return logits

In [2]:
from IPython.core.display import display_markdown
import torch.nn.functional as F

class NegativeLogLikelihood(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(NegativeLogLikelihood, self).__init__()

    def softmax(self, x):
        max = torch.max(x, dim=1, keepdim=True)[0]
        x_exp_shifted = torch.exp(x - max)
        return x_exp_shifted / torch.sum(x_exp_shifted, dim=1, keepdim=True)

    def forward(self, x, y):
        y = torch.eye(x.shape[1])[y].cuda()
        probs = self.softmax(x)
        return -torch.mean(torch.log(torch.sum(probs * y, dim=1)))

In [3]:
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import torch.backends.cudnn as cudnn

#Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

#hyperparameters
classes = ('plane', 'car', 'bird', 'cat', 'deer',
         'dog', 'frog', 'horse', 'ship', 'truck')

#torch.save(net, 'cnn_cifar.pth')
#Model
print('==> Building model..')
best_acc = 0  
start_epoch = 0  
device = 'cuda'
net = ConvolutionalModel(3, 32, 16, 3, 32, 3, 256, 128, 10)
net = net.to(device)

net = torch.nn.DataParallel(net)

criterion = NegativeLogLikelihood()
optimizer = optim.SGD(net.parameters(), lr=1e-2,
                      momentum=0.9, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
start_time = time.time()

#Training function
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))   
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print('Train -> Loss: %.3f | Acc: %.3f%%'
                  % (train_loss/(len(trainloader)), 100.*correct/total))
#Test function
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print('Test -> Loss: %.3f | Acc: %.3f%%'
                  % (test_loss/(len(testloader)), 100.*correct/total))

for epoch in range(start_epoch, start_epoch+30):
      train(epoch)
      test(epoch)
      scheduler.step()

==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
==> Building model..

Epoch: 0
Time elapsed: 0.00 min
Train -> Loss: 1.887 | Acc: 31.848%
Test -> Loss: 1.580 | Acc: 42.310%

Epoch: 1
Time elapsed: 0.23 min
Train -> Loss: 1.542 | Acc: 43.814%
Test -> Loss: 1.432 | Acc: 48.090%

Epoch: 2
Time elapsed: 0.46 min
Train -> Loss: 1.419 | Acc: 48.518%
Test -> Loss: 1.378 | Acc: 48.990%

Epoch: 3
Time elapsed: 0.69 min
Train -> Loss: 1.342 | Acc: 51.402%
Test -> Loss: 1.281 | Acc: 54.280%

Epoch: 4
Time elapsed: 0.93 min
Train -> Loss: 1.263 | Acc: 54.658%
Test -> Loss: 1.205 | Acc: 56.520%

Epoch: 5
Time elapsed: 1.17 min
Train -> Loss: 1.200 | Acc: 56.988%
Test -> Loss: 1.158 | Acc: 59.440%

Epoch: 6
Time elapsed: 1.40 min
Train -> Loss: 1.161 | Acc: 58.572%
Test -> Loss: 1.137 | Acc: 60.340%

Epoch: 7
Time elapsed: 1.64 min
Train -> Loss: 1.126 | Acc: 59.960%
Test -> Loss: 1.061 | Acc: 61.770%

Epoch: 8
Time elapsed: 1.87 min
Train -> Loss: 1.092 | Ac